### Recognise faces using KNN classifiers
  
  1. Load the training data (numpy arrays of all the persons)
         x-values are stored in numpy arrays
         y-values we will assign for each person.
  2. Read a video stream using OpenCV
  3. Extract faces out of it. (Create a test dataset)
  4. Use KNN algorithm to predict name of the person.
  5. Map the predicted id to name of the user
  6. Display the predictions on the screen - using a enclosing box and name 

In [1]:
import numpy as np
import cv2
import os

#### KNN Code

In [2]:
def distance(x1,x2):
    # if the points are as (1,2,3) and (4,5,6), then there dist will be given as: sqrt((1-4)**2 + (2-5)**2 + (3-6)**2 )
    # = sqrt(9+9+9) = sqrt(27)
    return np.sqrt(sum((x1-x2)**2))

def knn(X,Y,query_point,k=5):
    dist=[]
    m=X.shape[0]
    
    for i in range(m):
        d = distance(query_point,X[i])
        dist.append((d,Y[i])) # Storing the distance along with the label as a tuple
        
    #Now sorting the dist list based on the nearest d values
    dist = sorted(dist)
    
    #for K nearest distance values
    dist = dist[:k]
    
    dist = np.array(dist)
    
    #To count the no. of unique labels in in the list
    new_vals = np.unique(dist[:,1],return_counts=True)
    #print(new_vals)
    index = new_vals[1].argmax() #it gives the index of label having maximum count
    pred = new_vals[0][index]
    
    return pred

In [3]:
#Initialising camera
cap = cv2.VideoCapture(0)

#FaceDetection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

skip = 0
#file_name = input("Enter the name of the person : ")
dataset_path = './Face_Data/'   # It acts as the trained data
face_data = []
labels=[]  # Labels for given file

class_id = 0
names = {} #Mapping between id and name

# Data Preparation
for files in os.listdir(dataset_path):
    if files.endswith('.npy'):
        # Creating a label between class_id and its name
        names[class_id]=files[:-4]
        print("Loaded "+files)
        data_item = np.load(dataset_path+files)
        face_data.append(data_item)
        
        # Creating labels for the classes
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)
trained_face_dataset = np.concatenate(face_data,axis=0)
trained_face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(trained_face_dataset.shape)
print(trained_face_labels.shape)

Loaded HBDJivu.npy
Loaded Jivu.npy
Loaded Rajni.npy
Loaded Sanju.npy
Loaded Saurabh.npy
(45, 30000)
(45, 1)


#### Testing the classifier

In [4]:
while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces[-1:]:
        x,y,w,h = face
        
        offset  = 10 # A value taken for padding around the face
        face_sec = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_sec = cv2.resize(face_sec,(100,100))
        
        # Predicted output label
        out = knn(trained_face_dataset,trained_face_labels,face_sec.flatten())
        
        # Display name and the enclosing box around face
        predicted_name = names[int(out)]
        cv2.putText(frame,predicted_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    
    cv2.imshow("Faces",frame)
    
    KeyPressed = cv2.waitKey(1) & 0xFF
    if KeyPressed == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()